In [11]:
from etl.extract import Extract
from etl.transform import Transform
from etl.load import Load

In [12]:
extract = Extract()
transform = Transform()
load = Load()

In [13]:
# df_survey_flat = extract.read_api_json("p272193372940")
df_survey_flat = extract.read_api_json("p160991049881")

In [14]:
df_survey_flat.head()

,responseid,respid,status,interview_start,interview_end,lastchannel,lastdevicetype,lastrenderingmode,last_touched,first_question_on_last_page_displayed,...,q14_false,q14_100_other,q120_1,q120_3,q120_4,q120_2,q120_5,q120_970_other,happy_try_1,happy_try_2
0,1,1,screened,2023-03-10T01:24:50.467+00:00,2023-03-13T06:04:49.657+00:00,1,1,1,2023-03-13T06:04:49.707+00:00,Internal_Stop,...,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,100",,5,5,4,4,5,,5,5


In [15]:
#### Transform 
df_transform1 = transform.enunciado1(df_survey_flat)

In [16]:
df_transform1.head()

,responseid,respid,status,interview_start,interview_end,lastchannel,lastdevicetype,lastrenderingmode,last_touched,first_question_on_last_page_displayed,...,q14_false,q14_100_other,q120_1,q120_3,q120_4,q120_2,q120_5,q120_970_other,happy_try_1,happy_try_2
0,1,1,screened,2023-03-10T01:24:50.467+00:00,2023-03-13T06:04:49.657+00:00,1,1,1,2023-03-13T06:04:49.707+00:00,Internal_Stop,...,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,100",,5,5,4,4,5,,5,5


In [17]:
from utils import utils as u
import sqlalchemy as db

db = 'test_db'

tbl_name = "SurveyTest"

# u.sql_engine()

u.create_sql_tbl_schema(df_transform1, u.sql_conn(u.sql_engine()), db, tbl_name)

In [18]:
#### Load
load.load_sql(df_transform1, tbl_name)